In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-2326  GCE       4                                       RUNNING  us-central1-a


In [46]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 71.5 MB/s eta 0:00:0000:010:01


In [47]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.util import ngrams
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import google.auth
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 30 12:29 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bx_general_bucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [9]:
parquetFile = spark.read.parquet(*paths)

In [12]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [13]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [14]:
from inverted_index_gcp import InvertedIndex

In [29]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()

NUM_BUCKETS = 250

def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def tokenize(text, bigram=False):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """

    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    list_of_tokens = [stemmer.stem(token) for token in list_of_tokens]

    if bigram:
      bigrams = list(ngrams(list_of_tokens, 2))
      bigrams = [' '.join(bigram) for bigram in bigrams]
      return bigrams

    return list_of_tokens

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = tokenize(text)
  # YOUR CODE HERE
  result = []
  tf_dic = Counter(tokens)

  for token in tf_dic:
    tup = (token, (id, tf_dic[token]))
    result.append(tup)
  
  return result

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  res = []
  for item in unsorted_pl:
    res.append(item)

  return sorted(res, key=itemgetter(0))

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  rdd = postings.reduceByKey(lambda pl1, pl2: pl1 + pl2)
  rdd = rdd.map(lambda x: (x[0], len(x[1])))
  return rdd

def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, bucket_name):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  rdd = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])]))
  rdd2 = rdd.reduceByKey(lambda a,b: a+b)
  res = rdd2.map(lambda x: (InvertedIndex.write_a_posting_list((x[0], x[1]), bucket_name)))
  return res

In [32]:
def create_inverted_index(bucket_name, source, rares, bigram=False):
  ''' Takes a bucket name, source of text to index, number of rare words to filter, and a boolean that indicates whether we'll use bigram or not.
  Parameters:
  -----------
    bucket_name: string
      the name of the bucket that will store the index.
    source: string
      the source of text. title or body.
    rares: int
      a number to filter the words that appear in less than 'rares' documents.
    bigram: bool
      whether to tokenize by pairs or not.
  Returns:
  --------
    None - the function writes an inverted index to the bucket. 
  '''
  # sequence of transforamtions performed in exercise 3 to create and inverted index from corups
  documents = parquetFile.select(source, "id").rdd
  word_counts = documents.flatMap(lambda x: word_count(x[0], x[1]))
  postings = word_counts.groupByKey().mapValues(reduce_word_counts)
  postings_filtered = postings.filter(lambda x: len(x[1])>rares) # filtering rare words
  w2df = calculate_df(postings_filtered)
  posting_locs_list = partition_postings_and_write(postings_filtered, bucket_name).collect()
 
  super_posting_locs = defaultdict(list)
  for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
      continue
    with blob.open("rb") as f:
      posting_locs = pickle.load(f)
      for k, v in posting_locs.items():
        super_posting_locs[k].extend(v)
  
  # putting everything together
  inverted = InvertedIndex()
  inverted.posting_locs = super_posting_locs
  inverted.df = w2df.collectAsMap()
  inverted.write_index('.', f'{bucket_name}_index')
  index_src = f'{bucket_name}_index.pkl'
  index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
  !gsutil cp $index_src $index_dst

In [33]:
create_inverted_index(bucket_name='bx_title_index', source='title', rares=20, bigram=False)

Copying file://bx_title_index_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      


In [ ]:
create_inverted_index('bx_body_index', 'text', 50, bigram=False)

Copying file://bx_body_index_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 15.9 MiB/ 15.9 MiB]                                                
Operation completed over 1 objects/15.9 MiB.                                     


In [ ]:
create_inverted_index('bx_title_bigram_index', 'title', 20, bigram=True)

Copying file://bx_title_bigram_index_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      


In [ ]:
create_inverted_index('bx_body_bigram_index', 'text', 50, bigram=True)

Copying file://bx_body_bigram_index_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 15.9 MiB/ 15.9 MiB]                                                
Operation completed over 1 objects/15.9 MiB.                                     


In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE
  rdd = pages.map(lambda x: [(x[0], xs[0]) for xs in x[1]])
  rdd2 = rdd.flatMap(lambda x: [xs for xs in x])

  edges = rdd2.map(lambda x: (x[0], x[1])).distinct()
  vertices = rdd2.flatMap(lambda x: [x[0], x[1]]).distinct()
  vertices = vertices.map(lambda x: (x,))
  return edges, vertices

pages_links = parquetFile.select('id', 'anchor_text').rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://bx_general_bucket/pr', compression="gzip")

In [ ]:
# creating page views dictionary and saving it to bucket
from pathlib import Path

pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)

--2022-12-30 19:27:30--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.15MB/s    in 9m 12s  

2022-12-30 19:36:42 (4.32 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [55]:
import pickle
import google.auth
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.cloud import storage

# Authenticate and authorize your GCP account
credentials, project_id = google.auth.default()

# Serialize the object using pickle
data = pickle.dumps(wid2pv)

# Use the `googleapiclient` library to retrieve the project ID
service = build('cloudresourcemanager', 'v1', credentials=credentials)
request = service.projects().get(projectId=project_id)
response = request.execute()
project_name = response['name']

# Use the `google-cloud-storage` library to interact with GCS
client = storage.Client(project=project_name, credentials=credentials)

# Get the bucket
bucket_name = "bx_general_bucket"
bucket = client.bucket(bucket_name)

# Create a new GCS object with the serialized data
blob = bucket.blob("wid2pv.pkl")
blob.upload_from_string(data)
